In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip
/kaggle/input/word2vec-nlp-tutorial/sampleSubmission.csv
/kaggle/input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip
/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip


In [16]:
import zipfile
with zipfile.ZipFile('/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip','r') as z:
    with z.open('labeledTrainData.tsv') as f:
        df=pd.read_csv(f,sep='\t')

In [17]:
df.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [18]:
#all lib
from bs4 import BeautifulSoup
import string,time
from textblob import TextBlob
from nltk.corpus import stopwords
#all functions
stop_words=set(stopwords.words('english'))
def html_replacer(text):
    return BeautifulSoup(text,"html.parser").get_text()
def remove_punctuation(text):
    return text.translate(str.maketrans('','',string.punctuation))
def text_corrector(itext):
    text=TextBlob(itext)
    return str(text.correct())
def remove_hex(text):
    hexes=''.join(chr(i) for i in range(128,256))
    return text.translate(str.maketrans('','',hexes))
def remove_stopwords(text):
    return " ".join(word for word in text.split() if word not in stop_words)

In [19]:
#All preprocess
df['review']=df['review'].str.lower()
df['review']=df['review'].apply(html_replacer)
df['review']=df['review'].apply(remove_punctuation)
df['review']=df['review'].apply(remove_stopwords)
df['review']=df['review'].apply(remove_hex)

<ipython-input-18-18f801ca2afd>:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(text,"html.parser").get_text()


In [20]:
#Lemmatization
import spacy
from tqdm import tqdm
nlp=spacy.load('en_core_web_sm')
tqdm.pandas()
df['lemma']=[[token.lemma_ for token in doc] for doc in tqdm(nlp.pipe(df['review'],batch_size=50),total=len(df))]

100%|██████████| 25000/25000 [04:48<00:00, 86.60it/s] 


In [21]:
#tfidf vectorization
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(max_features=5000)
X=tfidf.fit_transform(df['lemma'].apply(lambda x:' '.join(x)))

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
X_train,X_test,y_train,y_test=train_test_split(X,df['sentiment'],test_size=0.2,random_state=42)
model=RandomForestClassifier()
model.fit(X_train,y_train)

RandomForestClassifier()

In [23]:
model.predict(X_test)

array([0, 1, 0, ..., 0, 0, 0])

In [25]:
import zipfile
with zipfile.ZipFile('/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip','r') as z:
    with z.open('testData.tsv') as f:
        df=pd.read_csv(f,sep='\t')

In [26]:
df.head()

,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


In [27]:
#all lib
from bs4 import BeautifulSoup
import string,time
from textblob import TextBlob
from nltk.corpus import stopwords
#all functions
stop_words=set(stopwords.words('english'))
def html_replacer(text):
    return BeautifulSoup(text,"html.parser").get_text()
def remove_punctuation(text):
    return text.translate(str.maketrans('','',string.punctuation))
def text_corrector(itext):
    text=TextBlob(itext)
    return str(text.correct())
def remove_hex(text):
    hexes=''.join(chr(i) for i in range(128,256))
    return text.translate(str.maketrans('','',hexes))
def remove_stopwords(text):
    return " ".join(word for word in text.split() if word not in stop_words)

In [28]:
#All preprocess
df['review']=df['review'].str.lower()
df['review']=df['review'].apply(html_replacer)
df['review']=df['review'].apply(remove_punctuation)
df['review']=df['review'].apply(remove_stopwords)
df['review']=df['review'].apply(remove_hex)

<ipython-input-27-18f801ca2afd>:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(text,"html.parser").get_text()


In [29]:
#Lemmatization
df['lemma']=[[token.lemma_ for token in doc] for doc in tqdm(nlp.pipe(df['review'],batch_size=50),total=len(df))]

100%|██████████| 25000/25000 [04:40<00:00, 89.00it/s] 


In [30]:
#tfidf vectorization
X_test=tfidf.transform(df['lemma'].apply(lambda x:' '.join(x)))

In [33]:
pred=model.predict(X_test)

In [35]:
ff=pd.DataFrame({'id':df['id'],'sentiment':pred})
ff.to_csv('submission.csv',index=False)